In [48]:
%load_ext autoreload
%autoreload 2

In [49]:
import numpy as np
VOCAB, EMBEDDING_MATRIX = np.load("data/vocab_glove_100d.npy", allow_pickle=True).item(), np.load("data/embedding_matrix_glove_100d.npy", allow_pickle=True)

In [50]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
import pytorch_lightning as pl
import torchmetrics
import torch.nn.functional as F

class SWEMNNClassifier(pl.LightningModule):
    def __init__(self, num_classes=3, hidden_dim=64, embedding_dim=100, learning_rate=1e-3):
        super().__init__()
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.activation = nn.LeakyReLU()
        self.linear2 = nn.Linear(hidden_dim, num_classes)
        self.model = nn.Sequential(self.linear1, self.activation, self.linear2)
        self.criterion = nn.BCEWithLogitsLoss()
        self.learning_rate = learning_rate
        self.train_accuracy = torchmetrics.Accuracy()
        self.val_accuracy = torchmetrics.Accuracy()
        self.train_precision = torchmetrics.Precision(num_classes=3, average='weighted')
        self.val_precision = torchmetrics.Precision(num_classes=3, average='weighted')
        self.train_recall = torchmetrics.Recall(num_classes=3, average='weighted')
        self.val_recall = torchmetrics.Recall(num_classes=3, average='weighted')
        self.train_f1 = torchmetrics.F1Score(num_classes=3, average='weighted')
        self.val_f1 = torchmetrics.F1Score(num_classes=3, average='weighted')
        self.save_hyperparameters()
    
    def forward(self, X):
        outputs = self.model(X)
        probs = F.sigmoid(outputs)
        return probs
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        outputs = self.model(X)
        train_loss = self.criterion(outputs, y.float())
        preds = self.forward(X)
        self.train_accuracy(preds, y)
        self.train_precision(preds, y)
        self.train_recall(preds, y)
        self.train_f1(preds, y)
        self.log("train_loss", train_loss, on_epoch=True)
        self.log('train_accuracy', self.train_accuracy, on_epoch=True)
        self.log('train_precision', self.train_precision, on_epoch=True)
        self.log('train_recall', self.train_recall, on_epoch=True)
        self.log('train_f1', self.train_f1, on_epoch=True)
        return train_loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        outputs = self.model(X)
        val_loss = self.criterion(outputs, y.float())
        preds = self.forward(X)
        self.val_accuracy(preds, y)
        self.val_precision(preds, y)
        self.val_recall(preds, y)
        self.val_f1(preds, y)
        self.log("val_loss", val_loss, on_epoch=True)
        self.log('val_accuracy', self.val_accuracy, on_epoch=True)
        self.log('val_precision', self.val_precision, on_epoch=True)
        self.log('val_recall', self.val_recall, on_epoch=True)
        self.log('val_f1', self.val_f1, on_epoch=True)
        return val_loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [51]:
train_data = torch.load("data/head_train_multi_no_pad_max.pt")
val_data = torch.load("data/head_val_multi_no_pad_max.pt")

In [52]:
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=128)

In [53]:
from pytorch_lightning.loggers import WandbLogger
import wandb

wandb_logger = WandbLogger(project="kogito-relation-matcher", name="swem_multi_label_nn")
model = SWEMNNClassifier(learning_rate=1e-4)
trainer = pl.Trainer(max_epochs=20, logger=wandb_logger)
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
wandb.finish()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mismayil (use `wandb login --relogin` to force relogin)



   | Name            | Type              | Params
-------------------------------------------------------
0  | linear1         | Linear            | 6.5 K 
1  | activation      | LeakyReLU         | 0     
2  | linear2         | Linear            | 195   
3  | model           | Sequential        | 6.7 K 
4  | criterion       | BCEWithLogitsLoss | 0     
5  | train_accuracy  | Accuracy          | 0     
6  | val_accuracy    | Accuracy          | 0     
7  | train_precision | Precision         | 0     
8  | val_precision   | Precision         | 0     
9  | train_recall    | Recall            | 0     
10 | val_recall      | Recall            | 0     
11 | train_f1        | F1Score           | 0     
12 | val_f1          | F1Score           | 0     
-------------------------------------------------------
6.7 K     Trainable params
0         Non-trainable params
6.7 K     Total params
0.027     Total estimated model params size (MB)


/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `Data

Epoch 19: 100%|██████████| 305/305 [00:06<00:00, 43.68it/s, loss=0.245, v_num=ptq3]



epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_accuracy_epoch,▁▄▆▇▇▇▇█████████████
train_accuracy_step,▂▁▂▄▅▆▆▇▇▇▅▇▆▇▇▆▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇█▇▇▇▇▆▇▇█
train_f1_epoch,▁▄▆▇▇▇▇█████████████
train_f1_step,▂▁▂▄▅▇▇▇▇▇▅▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▆▇▇█
train_loss_epoch,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▅▄▃▃▃▂▂▄▂▂▂▂▃▂▂▂▂▂▃▂▂▂▁▁▂▂▂▂▁▂▂▂▂▂▂▂▁
train_precision_epoch,▁▅▆▇▇▇██████████████
train_precision_step,▁▁▂▃▅▆▆▆▆▆▄▇▇▇▆▆▆▇▇▇▇▆▆▇▆▇▆▇▆▆▇█▇▆▇▆▆▇▆▇
train_recall_epoch,▁▄▆▆▇▇▇▇▇███████████
train_recall_step,▁▁▂▂▄▇▇█▆▆▅▇▅▇▇▅▆▆▇▇▆▆█▇▇▇█▆▇▆▇▇▆▆▇▇▆▇▆█


In [4]:
from relation_modeling_utils import load_data, HeadDataset
from torch.utils.data import DataLoader

test_df = load_data("data/atomic2020_data-feb2021/test.tsv", multi_label=True)
test_data = HeadDataset(test_df, vocab=VOCAB, embedding_matrix=EMBEDDING_MATRIX, apply_pooling=True, pooling="max")
test_dataloader = DataLoader(test_data, batch_size=len(test_data))

In [5]:
test_df.head()

,text,label
0,PersonX abuses PersonX's power,"[0, 1, 1]"
1,PersonX accepts PersonY's apology,"[0, 1, 1]"
2,PersonX accepts ___ in payment,"[0, 1, 1]"
3,PersonX accidentally kicked,"[0, 1, 1]"
4,PersonX accidentally kicked ___,"[0, 1, 1]"


In [7]:
len(test_data.texts), len(test_df)

(6469, 6569)

In [11]:
import torch
model = torch.load('swem_nn_model.bin')

In [12]:
X, y = next(iter(test_dataloader))
preds = model.forward(X)

/Users/mismayil/opt/anaconda3/envs/kogito/lib/python3.8/site-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [13]:
import torchmetrics
test_accuracy = torchmetrics.Accuracy()
test_precision = torchmetrics.Precision(num_classes=3, average="weighted")
test_recall = torchmetrics.Recall(num_classes=3, average="weighted")
test_f1 = torchmetrics.F1Score(num_classes=3, average="weighted")
print(f'Test accurayc={test_accuracy(preds, y).item():.3f}, precision={test_precision(preds, y).item():.3f}, recall={test_recall(preds, y).item():.3f}, f1={test_f1(preds, y).item():.3f}')

Test accurayc=0.843, precision=0.818, recall=0.935, f1=0.861


In [15]:
preds.detach().tolist()

[[0.01173484232276678, 0.7562248110771179, 0.956723690032959],
 [0.01389793399721384, 0.7243813276290894, 0.9530561566352844],
 [0.0037115702871233225, 0.7532876133918762, 0.9834098815917969],
 [0.07428852468729019, 0.7708461284637451, 0.794985294342041],
 [0.012255345471203327, 0.8351738452911377, 0.9524644613265991],
 [0.019262975081801414, 0.8001779317855835, 0.9177660346031189],
 [0.013090131804347038, 0.8877256512641907, 0.954416811466217],
 [0.0016744000604376197, 0.8531801700592041, 0.9939131140708923],
 [0.0013083158992230892, 0.8636524677276611, 0.9933453798294067],
 [0.01387664582580328, 0.7723642587661743, 0.9574757814407349],
 [0.2156227082014084, 0.5196855068206787, 0.6124313473701477],
 [0.6392898559570312, 0.3483458459377289, 0.2832302153110504],
 [0.016336780041456223, 0.7625696659088135, 0.960696280002594],
 [0.01774066872894764, 0.7293390035629272, 0.9423817992210388],
 [0.01774066872894764, 0.7293390035629272, 0.9423817992210388],
 [0.022432420402765274, 0.6779159307

In [29]:
test_confusion = torchmetrics.ConfusionMatrix(num_classes=3, multilabel=True)
confusion_matrix = test_confusion(preds, y)

In [30]:
confusion_matrix

tensor([[[4519,  147],
         [ 143, 1660]],

        [[1779,  271],
         [ 275, 4144]],

        [[1838, 2060],
         [ 151, 2420]]])

In [41]:
import pandas as pd
pred_df = pd.DataFrame({'texts': test_data.texts, 'labels': test_data.labels.tolist(), 'probs': preds.detach().tolist()})
pred_df['preds'] = pred_df.probs.apply(lambda p: (np.array(p) >= 0.5).astype(int).tolist())

In [42]:
pred_df.head()

,texts,labels,probs,preds
0,PersonX abuses PersonX's power,"[0, 1, 1]","[0.01173484232276678, 0.7562248110771179, 0.95...","[0, 1, 1]"
1,PersonX accepts PersonY's apology,"[0, 1, 1]","[0.01389793399721384, 0.7243813276290894, 0.95...","[0, 1, 1]"
2,PersonX accepts ___ in payment,"[0, 1, 1]","[0.0037115702871233225, 0.7532876133918762, 0....","[0, 1, 1]"
3,PersonX accidentally kicked,"[0, 1, 1]","[0.07428852468729019, 0.7708461284637451, 0.79...","[0, 1, 1]"
4,PersonX accidentally kicked ___,"[0, 1, 1]","[0.012255345471203327, 0.8351738452911377, 0.9...","[0, 1, 1]"


In [43]:
pred_df['matches'] = pred_df.apply(lambda row: (np.array(row.labels) * np.array(row.preds)).sum().tolist(), axis=1)

In [46]:
len(pred_df[pred_df['matches'] < 1])

383

In [47]:
pred_df[pred_df['matches'] < 1].head()

,texts,labels,probs,preds,matches
11,PersonX acts upon PersonY,"[0, 1, 1]","[0.6392898559570312, 0.3483458459377289, 0.283...","[1, 0, 0]",0
75,PersonX attention whores,"[0, 1, 1]","[0.6172634363174438, 0.3905506730079651, 0.234...","[1, 0, 0]",0
136,PersonX bosses PersonY around,"[0, 1, 1]","[0.39106833934783936, 0.32732418179512024, 0.4...","[0, 0, 0]",0
173,PersonX bullies at school,"[0, 1, 1]","[0.3215026557445526, 0.3772876262664795, 0.448...","[0, 0, 0]",0
177,PersonX burns badly,"[0, 1, 1]","[0.4727492332458496, 0.47334837913513184, 0.33...","[0, 0, 0]",0


In [ ]:
torch.save(model, "swem_nn_model.bin")

In [ ]:
from relation_modeling_utils import text_to_embedding

text = "death is a bad event"
embedding = text_to_embedding(text, vocab=VOCAB, embedding_matrix=EMBEDDING_MATRIX)
text_pred = model.forward(torch.tensor(embedding).view((1, -1)))
text_pred

In [ ]:
x = torch.tensor([[0.1, 0.2, 0.5], [0.9, 0.05, 0.05], [0.3, 0.3, 0.4], [0.1, 0.2, 0.7]])
y = torch.tensor([[0, 1, 1], [1, 0, 0], [0, 0, 1], [1, 0, 1]])
accuracy = torchmetrics.Accuracy()
accuracy(x, y)